In [1]:
#Import all the dependencies
import gensim
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
import numpy as np

import pandas as pd

MODEL_NAME = "doc2vec.model"
SIZE = 25
WINDOW_SIZE = 3

POS_TRAIN_PATH = "aclImdb/train/pos/"
NEG_TRAIN_PATH = "aclImdb/train/neg/"
POS_TEST_PATH = "aclImdb/test/pos/"
NEG_TEST_PATH = "aclImdb/test/neg/"

In [2]:
train_features = []
train_labels = []
test_features = []
test_labels = []

In [3]:
filenames = [f for f in listdir(POS_TRAIN_PATH) if f.endswith('.txt')]
for filename in filenames:
    data = open(POS_TRAIN_PATH + filename).read()
    train_features.append(data)
    train_labels.append(1)
    
filenames = [f for f in listdir(NEG_TRAIN_PATH) if f.endswith('.txt')]
for filename in filenames:
    data = open(NEG_TRAIN_PATH + filename).read()
    train_features.append(data)
    train_labels.append(0)

In [4]:
filenames = [f for f in listdir(POS_TEST_PATH) if f.endswith('.txt')]
for filename in filenames:
    data = open(POS_TEST_PATH + filename).read()
    test_features.append(data)
    test_labels.append(1)
    
filenames = [f for f in listdir(NEG_TEST_PATH) if f.endswith('.txt')]
for filename in filenames:
    data = open(NEG_TEST_PATH + filename).read()
    test_features.append(data)
    test_labels.append(0)

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
stopword_set = set(stopwords.words('english'))

def nlp_clean(data):
   new_data = []
   for d in data:
      new_str = d.lower()
      dlist = tokenizer.tokenize(new_str)
      dlist = list(set(dlist).difference(stopword_set))
      new_data.append(dlist)
        
   return new_data

train_features = nlp_clean(train_features)
test_features = nlp_clean(test_features)

In [6]:
mean_len = 0
bow = []
for f in train_features:
    mean_len = mean_len + len(f)
    bow.extend(f)
    
for f in test_features:
    mean_len = mean_len + len(f)
    bow.extend(f)
    
mean_len = np.int32(mean_len / (len(bow)))
bow = np.array(bow)

In [7]:
mean_len = np.int16(np.ceil(mean_len / 100) * 100)

In [8]:
mean_len

100

In [9]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(bow)
print(len(encoder.classes_))

101791


In [10]:
x_train = []
x_test = []

counter = 0
for f in train_features:
    r = np.array(f)
    r = encoder.transform(r)
    x_train.append(r)
    counter = counter + 1
    if counter % 100 == 0:
        print(counter)

counter = 0
for f in test_features:
    r = np.array(f)
    r = encoder.transform(r)
    x_test.append(r)
    counter = counter + 1
    if counter % 100 == 0:
        print(counter)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [11]:
x_train_copy = x_train.copy()
x_test_copy = x_test.copy()

In [13]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train_copy, maxlen=mean_len)
x_test = sequence.pad_sequences(x_test_copy, maxlen=mean_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

/Users/yogesh/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [14]:
columns = [str(x) for x in range(mean_len)]
columns.append("Sentiment")

In [15]:
train_data = None

for idx in range(len(x_train)):
    features = x_train[idx]
    label = np.array([train_labels[idx]], dtype=np.int16)
    row = np.array([np.concatenate((features, label), axis = 0)])
    
    if train_data is None:
        train_data = row
    else:
        train_data = np.concatenate((train_data, row), axis=0)
    

In [16]:
np.shape(train_data)

(25000, 101)

In [17]:
test_data = None

for idx in range(len(x_test)):
    features = x_test[idx]
    label = np.array([test_labels[idx]], dtype=np.int16)
    row = np.array([np.concatenate((features, label), axis = 0)])
    
    if test_data is None:
        test_data = row
    else:
        test_data = np.concatenate((test_data, row), axis=0)
        

In [18]:
np.shape(test_data)

(25000, 101)

In [19]:
train_data = pd.DataFrame(columns=columns, data=train_data)
test_data = pd.DataFrame(columns=columns, data=test_data)

In [20]:
train_data = train_data.sample(frac=1)
test_data = test_data.sample(frac=1)

In [21]:
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Sentiment
22074,12030,25331,77550,29530,100950,100266,60307,94367,60162,58112,...,6715,68067,19381,95382,70521,79657,8895,33267,91198,0
3941,77551,66326,26872,8372,21910,38042,94564,89805,41134,60162,...,42198,15660,78398,70521,9278,29662,56979,100133,33267,1
18316,0,0,0,0,0,0,0,0,0,0,...,51596,12303,54934,18591,34435,81321,79657,33267,91198,0
10129,0,0,0,0,0,0,0,0,0,0,...,73259,4943,34444,79657,52669,26476,67364,55571,24613,1
4054,24351,52701,38067,26810,52608,5363,51639,78674,56036,34304,...,15105,54934,55379,28025,91457,81321,79657,100133,91198,1


In [22]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 22074 to 4897
Columns: 101 entries, 0 to Sentiment
dtypes: int32(101)
memory usage: 9.8 MB


In [23]:
test_data.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Sentiment
4918,31523,97993,83719,7854,51372,90328,2127,96036,82008,58112,...,7175,2208,18377,9167,8502,6821,42698,69134,56965,1
6736,0,0,0,0,0,0,0,0,0,0,...,96858,30795,64810,184,1088,2378,73295,57381,78622,1
13708,0,0,0,0,0,0,0,0,0,0,...,37710,74317,44001,83587,81362,59322,66714,89905,55571,0
18379,23440,34874,84045,84637,100777,97429,55026,52710,64123,15702,...,35137,15105,60279,50629,5412,37918,81321,25372,43233,0
14417,0,0,0,0,0,0,0,0,0,0,...,10550,67007,53005,38067,70377,50514,24180,57381,89286,0


In [24]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 4918 to 11044
Columns: 101 entries, 0 to Sentiment
dtypes: int32(101)
memory usage: 9.8 MB


In [25]:
train_data.to_csv("train_data_lstm.csv", index=False)
test_data.to_csv("test_data_lstm.csv", index=False)